# Setup

In [146]:
import sqlite3
import time
import importlib
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
import plotly as ply
import re
import dopetracks_summary.data_prep.spotify_db_manager as sdm
import dopetracks_summary.data_prep.prepare_data_main as pdm
import dopetracks_summary.data_prep.data_enrichment as de
import dopetracks_summary.create_spotify_playlist as csp
import dopetracks_summary.utility_functions as uf
import logging
import requests
import json
import numpy as np

db_path = "/Users/nmarks/Library/Messages/chat.db"  # Adjust path as needed
conn = sqlite3.connect(db_path)

importlib.reload(sdm)
importlib.reload(uf)
importlib.reload(csp)
importlib.reload(pdm)


<module 'dopetracks_summary.data_prep.prepare_data_main' from '/Users/nmarks/root_code_repo/dopetracks_summary/src/dopetracks_summary/data_prep/prepare_data_main.py'>

# 1 Pull + clean data

In [147]:
data = pdm.pull_and_clean_messages()

2025-01-01 16:42:28 - INFO - 
----------------------------------------------------------
[1] Pulling and cleaning iMessage data
----------------------------------------------------------

2025-01-01 16:42:28 - INFO - Connecting to the database...
2025-01-01 16:42:28 - INFO - Database connection established. Time taken: 0.00s
2025-01-01 16:42:28 - INFO - Pulling data...


Database connection established.


2025-01-01 16:42:36 - INFO - Data successfully pulled! Time taken: 8.04s
2025-01-01 16:42:36 - INFO - Cleaning data...
2025-01-01 16:42:37 - INFO - Data successfully cleaned! Time taken: 0.65s
2025-01-01 16:42:37 - INFO - Enriching data...
2025-01-01 16:43:16 - INFO - Data successfully enriched! Time taken: 39.10s
2025-01-01 16:43:16 - INFO - Importing contact info...
2025-01-01 16:43:16 - INFO - Successfully pulled contact info! Time taken: 0.01s
2025-01-01 16:43:16 - INFO - Messages database connection (chat.db) closed.
2025-01-01 16:43:16 - INFO - 
----------------------------------------------------------
[1] COMPLETED Pulling and cleaning iMessage data
    (Time taken: 47.79s)"
----------------------------------------------------------



# Top Sender

In [291]:
data['messages'][[
    'message_id','chat_id', 'final_text', 'sender_handle_id', 'reaction_type', 'date'
]].groupby('reaction_type').agg(
        messages_sent=('sender_handle_id', 'size'),
        first_message_date=('date', 'min'),
        last_message_date=('date', 'min')
    ).fillna({'messages_sent': 0, 'loves_sent': 0, 'links_sent': 0})

,messages_sent,first_message_date,last_message_date
reaction_type,,,
Disliked,384,2016-12-30,2016-12-30
Emphasized,3819,2017-01-10,2017-01-10
Laughed,4285,2016-11-07,2016-11-07
Liked,5859,2016-11-07,2016-11-07
Loved,9102,2017-04-01,2017-04-01
Questioned,250,2017-07-14,2017-07-14
Removed dislike,3,2017-08-07,2017-08-07
Removed emphasis,9,2017-07-14,2017-07-14
Removed heart,21,2017-08-28,2017-08-28


In [301]:
from dopetracks_summary import generate_summary_stats as gss
importlib.reload(gss)

summary_data = gss.main(data)
summary_data['user_stats'].sort_values('links_sent', ascending=False)

,contact_info,first_name,last_name,messages_sent,non_reaction_messages,loves_sent,likes_sent,hahas_sent,dislikes_sent,questions_sent,emphasized_sent,links_sent,first_message_date,last_message_date,handles
0,+15037037962,Aaron,Poplack,266.0,194.0,53.0,3.0,7.0,1.0,0.0,8.0,55.0,2024-01-04,2024-12-13,"[18, 360]"
1,+15074407296,Seth,Johnson,146.0,94.0,31.0,1.0,11.0,0.0,0.0,9.0,15.0,2024-01-04,2024-12-18,"[33, 11]"
5,+15037036837,Chet,unknown,63.0,53.0,5.0,4.0,1.0,0.0,0.0,0.0,10.0,2024-01-25,2024-12-20,"[40, 364]"
2,+15038079404,Ari,Kogan,104.0,82.0,12.0,2.0,2.0,1.0,0.0,5.0,8.0,2024-01-04,2024-12-09,"[15, 5]"
8,+16616006144,Lindsey,Rockwell,41.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2024-01-05,2024-11-17,"[35, 358]"
11,+19715064236,Jake,Rockwell,29.0,27.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,2024-01-09,2024-12-09,"[27, 144]"
6,+15413376354,Melissa,Bernstein,58.0,34.0,19.0,0.0,3.0,0.0,0.0,2.0,3.0,2024-01-15,2024-11-20,"[39, 365]"
4,+15033199126,max,barenberg,80.0,69.0,6.0,2.0,1.0,0.0,0.0,2.0,2.0,2024-01-09,2024-12-26,"[16, 6]"
7,+15035040080,Nick,Marks,50.0,26.0,8.0,2.0,8.0,0.0,0.0,6.0,2.0,2024-01-15,2024-11-20,[1]
9,+12678041557,Tim,Israel,36.0,30.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,2024-01-15,2024-12-03,"[25, 362]"


In [241]:
import dopetracks_summary.data_prep.import_contact_info as ici
import importlib
from datetime import datetime
import pandas as pd

importlib.reload(ici)
contacts = ici.main()
contacts['phone_number'] = contacts['phone_number'].apply(lambda x: '+1' + x if isinstance(x, str) else x)

# Get dope tracks members by handle_id
dope_tracks_members = data['chat_handle_join'][data['chat_handle_join']['chat_id'].isin([16, 286])]['handle_id'].unique()
dope_tracks_members_df = pd.DataFrame(dope_tracks_members, columns=['handle_id'])
me = pd.DataFrame({'handle_id': [1]})
dope_tracks_members_df = pd.concat([dope_tracks_members_df, me], ignore_index=True)

# Merge handle data
dope_tracks_members_df = pd.merge(
    dope_tracks_members_df,
    data['handles'][['handle_id', 'contact_info']],
    on='handle_id',
    how='left'
).sort_values('contact_info')

# Get activity by handle_id
dopetracks_activity = data['messages'][
    data['messages']['chat_id'].isin([16, 286]) &
    (data['messages']['date'] >= datetime(2024, 1, 1)) & 
    (data['messages']['date'] < datetime(2025, 1, 1))
].groupby('sender_handle_id').agg(
    messages_sent=('sender_handle_id', 'size'),
    loves_sent=('reaction_type', lambda x: x.eq('Loved').sum()),
    links_sent=('spotify_song_links', lambda x: sum(1 for links in x if len(links) > 0))
).fillna({'messages_sent': 0, 'loves_sent': 0, 'links_sent': 0}).reset_index()

# Merge activity with handles
dopetracks_handle_activity = pd.merge(
    dope_tracks_members_df,
    dopetracks_activity,
    left_on="handle_id",
    right_on="sender_handle_id",
    how="left"
).fillna({'messages_sent': 0, 'loves_sent': 0, 'links_sent': 0}).sort_values('messages_sent', ascending=False)

dopetracks_handle_activity_with_names = pd.merge(
    dopetracks_handle_activity,
    contacts,
    left_on="contact_info",
    right_on="phone_number",
    how="left"
).fillna({'first_name': "unknown", 'last_name': "unknown"})


# Merge with contacts and group by name
dopetracks_handle_activity_with_names.groupby(["contact_info", "first_name", "last_name"]).agg(
    messages_sent=('messages_sent', 'sum'),
    loves_sent=('loves_sent', 'sum'),
    links_sent=('links_sent', 'sum')
).fillna({'messages_sent': 0, 'loves_sent': 0, 'links_sent': 0}).sort_values('messages_sent', ascending=False).reset_index()


,contact_info,first_name,last_name,messages_sent,loves_sent,links_sent
0,+15037037962,Aaron,Poplack,266.0,53.0,55.0
1,+15074407296,Seth,Johnson,146.0,31.0,15.0
2,+15038079404,Ari,Kogan,104.0,12.0,8.0
3,+15414907101,Kristi,Thomson,88.0,10.0,2.0
4,+15033199126,max,barenberg,80.0,6.0,2.0
5,+15037036837,Chet,unknown,63.0,5.0,10.0
6,+15413376354,Melissa,Bernstein,58.0,19.0,3.0
7,+15035040080,Nick,Marks,50.0,8.0,2.0
8,+16616006144,Lindsey,Rockwell,41.0,0.0,7.0
9,+12678041557,Tim,Israel,36.0,6.0,2.0


In [132]:
top_link_sender = data['messages'][
    (data['messages']['chat_id'].isin([16, 286])) &
    (data['messages']['date'] > datetime(2024, 1, 1)) & 
    (data['messages']['spotify_song_links'].apply(len) > 0) & 
    (data['messages']['chat_name'].apply(lambda x: isinstance(x, str) and "dope tracks" in x.lower()))
    ].groupby('sender_handle_id').size().sort_values(ascending=False).reset_index(name='size')


pd.merge(top_link_sender, data['handles'][['handle_id','contact_info']], left_on = 'sender_handle_id', right_on= 'handle_id', how = 'left')

,sender_handle_id,size,handle_id,contact_info
0,18,55,18,+15037037962
1,33,13,33,+15074407296
2,40,9,40,+15037036837
3,15,8,15,+15038079404
4,35,7,35,+16616006144
5,27,4,27,+19715064236
6,39,3,39,+15413376354
7,16,2,16,+15033199126
8,17,2,17,+12069482125
9,25,2,25,+12678041557


# Appendix

In [157]:
# data['messages']['num_spotify_links'] = data['messages']['all_spotify_links'].apply(len)

with pd.option_context('display.max_colwidth', None, 'display.max_columns', None):
    uf.display_scrollable(

        data['messages'][
                            (data['messages']['chat_id'].isin([16, 286])) &
                            (data['messages']['date'] > datetime(2024, 1, 1)) & 
                            # (data['messages']['spotify_song_links'].apply(len) > 0) & 
                            (data['messages']['chat_name'].apply(lambda x: isinstance(x, str) and "dope tracks" in x.lower()))

                          ]
        [['date_utc', 'reaction_type', 'chat_id','sender_handle_id', 'final_text', 'chat_name','spotify_song_links']]
    )

date_utc,reaction_type,chat_id,sender_handle_id,final_text,chat_name,spotify_song_links
2024-12-26 14:16:33,no-reaction,286.0,6,https://open.spotify.com/album/0m8G0dSYf4rc8SRftPjZXU?si=xxACGM_JTVOEQqjXNzIzJQ,Dope tracks (🔥🎧),[]
2024-12-20 13:58:36,no-reaction,286.0,364,https://on.soundcloud.com/Aoh95D2kjpou89VH9,Dope tracks (🔥🎧),[]
2024-12-20 13:57:14,no-reaction,286.0,364,,Dope tracks (🔥🎧),[]
2024-12-20 13:57:07,no-reaction,286.0,364,https://open.spotify.com/track/0s7MuB2jO2wukzMq9w9pG6?si=Lbu88E2cTd2hq4DuutVt9Q,Dope tracks (🔥🎧),[https://open.spotify.com/track/0s7MuB2jO2wukzMq9w9pG6?si=Lbu88E2cTd2hq4DuutVt9Q]
2024-12-18 10:04:14,no-reaction,286.0,11,https://open.spotify.com/track/3GPeN4bDQfVN7t4LxXQTs7?si=YsP2FZnITTeLp52oKk2xnA,Dope tracks (🔥🎧),[https://open.spotify.com/track/3GPeN4bDQfVN7t4LxXQTs7?si=YsP2FZnITTeLp52oKk2xnA]
2024-12-13 18:30:34,no-reaction,286.0,367,En route!,Dope tracks (🔥🎧),[]
2024-12-13 14:01:48,no-reaction,286.0,11,I assume they are the scientific terms for Raw & Unpasteurized,Dope tracks (🔥🎧),[]
2024-12-13 14:01:10,Emphasized,286.0,147,"Emphasized “Most of it is.....you guessed it, adulterated. It'…”",Dope tracks (🔥🎧),[]
2024-12-13 14:00:44,Emphasized,286.0,11,Emphasized “It's an extremely expensive type of honey used for…”,Dope tracks (🔥🎧),[]
2024-12-13 13:55:51,no-reaction,286.0,11,I don't see the words Dumb or Sticky on this bottle,Dope tracks (🔥🎧),[]
